## MLPRegressor Batch és Online

Különböző beállításokkal,

Baromira nem mindegy, hogy milyen beállításokat használunk, néha ezek hatására képtelen tanulni.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
x = np.arange(0, 600, 1)

y = np.sin(x/180) + np.sin(x/60)

x = x.reshape(-1,1)
y = y

## Tanh (logistic, relu)

## SGD (adam)

## Warm_start (False, True)

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(3, ),
                   activation='tanh', 
                   solver='sgd',
                   batch_size='auto',
                   learning_rate_init=0.01,
                   max_iter=2000,
                   shuffle=False,
                   random_state=1,
                   momentum=0.9,
                   nesterovs_momentum=True,
                   early_stopping=True,
                   n_iter_no_change=2000)

## Ilyen az amikor x és y is (-1,1) közé van normalizálva

In [ ]:
scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1
mlp.fit(x_scaled, y_scaled)
predicted = mlp.predict(x_scaled)
plt.scatter(x_scaled, y_scaled)
plt.plot(x_scaled, predicted, c='black')
plt.show()

## Tehát a fentiekből az látszik

#### Hogy a 'logistic' esetében nem képes megtalálni a megfelelő összefüggést.

Azonban alább elvégeztem a következő kísérletet és az derült ki,<br>
hogy az 'sgd' optimizer nem volt képes megtalálni az összefüggést.<br>
Ha az optimizer 'adam' akkor a ha mindkét változót (-1,1) tartományba<br>
normalizálom, akkor szintén jó megoldás születik.

Ebből azt a következtetést vonom le, hogy 'logistic' esetén is az a célravezető<br>
ha mindkét válotzót a (-1,1) tartományba transzformálom.

## Bemutató

In [ ]:
! rm *.png
! rm *.gif

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1


for i in range(0, 1000, 50):
  print('#i = ', i)
  if( i != 0 ):
    mlp = MLPRegressor(hidden_layer_sizes=(10, 5),
                      activation='tanh', 
                      solver='sgd',
                      batch_size='auto',
                      learning_rate_init=0.01,
                      max_iter=i,
                      shuffle=True,
                      random_state=1,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      n_iter_no_change=2000)

    mlp.fit(x_scaled, y_scaled)
    predicted = mlp.predict(x_scaled)
    plt.scatter(x_scaled, y_scaled)
    plt.plot(x_scaled, predicted, c='black')
    plt.title('#i = {0:03}'.format(i))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()

from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

## Ez így rossz Warm_start (False, True)

Mert a for cikluson belül példányosítom az ``` MLPRegressor ``` ezért nem is működik.



In [ ]:
scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1


for i in range(0, 600, 100):
  print('#i = ', i)
  if( i != 0 ):
    mlp1 = MLPRegressor(hidden_layer_sizes=(3, ),
                      activation='tanh', 
                      solver='sgd',
                      batch_size='auto',
                      learning_rate_init=0.05,
                      max_iter=i,
                      shuffle=False,
                      random_state=1,
                      warm_start=False,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      n_iter_no_change=2000)
    
    mlp2 = MLPRegressor(hidden_layer_sizes=(3, ),
                      activation='tanh', 
                      solver='sgd',
                      batch_size='auto',
                      learning_rate_init=0.05,
                      max_iter=i,
                      shuffle=False,
                      random_state=1,
                      warm_start=True,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      n_iter_no_change=2000)

    mlp1.fit(x_scaled, y_scaled)
    predicted1 = mlp1.predict(x_scaled)

    mlp2.fit(x_scaled, y_scaled)
    predicted2 = mlp2.predict(x_scaled)

    fig, ax = plt.subplots(1,2, figsize=(12,4))
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted1, c='black')
    ax[1].scatter(x_scaled, y_scaled)
    ax[1].plot(x_scaled, predicted2, c='black')
    plt.show()

## Ez így jó Warm_start (False, True)

Mert a for cikluson kívűl példányosítom az ``` MLPRegressor ```

In [ ]:
scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1

mlp1 = MLPRegressor(hidden_layer_sizes=(3, ),
                  activation='tanh', 
                  solver='sgd',
                  batch_size='auto',
                  learning_rate_init=0.05,
                  max_iter=i,
                  shuffle=False,
                  random_state=1,
                  warm_start=False,
                  momentum=0.9,
                  nesterovs_momentum=True,
                  early_stopping=True,
                  n_iter_no_change=2000)

mlp2 = MLPRegressor(hidden_layer_sizes=(3, ),
                  activation='tanh', 
                  solver='sgd',
                  batch_size='auto',
                  learning_rate_init=0.05,
                  max_iter=i,
                  shuffle=False,
                  random_state=1,
                  warm_start=True,
                  momentum=0.9,
                  nesterovs_momentum=True,
                  early_stopping=True,
                  n_iter_no_change=2000)

for i in range(0, 600, 100):
  print('#i = ', i)
  if( i != 0 ):

    mlp1.fit(x_scaled, y_scaled)
    predicted1 = mlp1.predict(x_scaled)

    mlp2.fit(x_scaled, y_scaled)
    predicted2 = mlp2.predict(x_scaled)

    fig, ax = plt.subplots(1,2, figsize=(12,4))
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted1, c='black')
    ax[1].scatter(x_scaled, y_scaled)
    ax[1].plot(x_scaled, predicted2, c='black')
    plt.show()

## Online learning

## Kívül van normálva

In [ ]:
mlp.best_loss_

In [ ]:
mlp.loss

In [ ]:
mlp.loss_

In [ ]:
mlp.loss_curve_

In [ ]:
mlp.partial_fit

In [ ]:
plt.plot(mlp.loss_curve_)

In [ ]:
! rm *.png
! rm *.gif


# mindíg az aktuális értéket normáljuk és nem előre

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one = MinMaxScaler(feature_range=(-1,1))

################################################################################
# A lényeg, hogy nehogy  a forciklusba tegyük a példányosítást, mert akkor
# minden egyes alkalommal újra új példány jön létre és elfelejti a korábbi
# súllyokat amiket megtanult
################################################################################

mlp = MLPRegressor(hidden_layer_sizes=(10, 5),
                  activation='tanh',              # tanh, logistic, relu
                  solver='sgd',                   # adam, sgd, lbfgs
                  batch_size='auto',
                  learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                  max_iter=1,                     # Fontos
                  shuffle=False,                  # Fontos
                  random_state=1,
                  warm_start=True,                # Fontos ?
                  momentum=0.9,
                  nesterovs_momentum=False,
                  early_stopping=False,           # Fontos ?
                  verbose=1,
                  n_iter_no_change=1000)


for i in range(0, 500, 1):
  print('#i = ', i)
  if( i > 2 ):

    scaler_x_one.fit(x)
    scaler_y_one.fit(y.reshape(-1,1))
    xs1 = scaler_x_one.transform(x)
    ys1 = scaler_y_one.transform(y.reshape(-1,1))
    print('xs1.min = ', xs1.min())
    print('xs1.max = ', xs1.max())
    print('ys1.min = ', ys1.min())
    print('ys1.max = ', ys1.max())
    print('ys1.shape = ', ys1.shape)
    x_scaled = xs1
    y_scaled = ys1

    x_part = x_scaled[:i,:]
    y_part = y_scaled[:i,:]

    mlp.fit(x_part, y_part)
    predicted = mlp.predict(x_part)
    plt.scatter(x_part, y_part)
    plt.plot(x_part, predicted, c='black')
    plt.title('#i = ' + str(i) + ' r2 = {:.3f}'.format(round(r2,3)) + ' loss = {:.3f}'.format(round(mlp.loss_, 3)))
    plt.ylim((-1.2, 1.2))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()


from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

## Ugyan ez csak az illeszkedés mértékét is plottolja

## Kívül van normálva

Ami itt érdekes lehet, mi van ha nem egyesévél léptetem a for ciklust, az adatokat? (1, 2, 5, 10)

### ToDo:

Kipróbálni a  ```partial_fit``` methodust a ```fit``` helyett.



In [ ]:
! rm *.png
! rm *.gif


# mindíg az aktuális értéket normáljuk és nem előre

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one = MinMaxScaler(feature_range=(-1,1))

################################################################################
# A lényeg, hogy nehogy  a forciklusba tegyük a példányosítást, mert akkor
# minden egyes alkalommal újra új példány jön létre és elfelejti a korábbi
# súllyokat amiket megtanult
################################################################################

mlp = MLPRegressor(hidden_layer_sizes=(10, 5),
                  activation='tanh',              # tanh, logistic, relu
                  solver='sgd',                   # adam, sgd, lbfgs
                  batch_size='auto',
                  learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                  max_iter=1,                     # Fontos
                  shuffle=False,                  # Fontos
                  random_state=1,
                  warm_start=True,                # Fontos ?
                  momentum=0.9,
                  nesterovs_momentum=False,
                  early_stopping=False,           # Fontos ?
                  verbose=1,
                  n_iter_no_change=1000)

r2_history = [0]

for i in range(0, 500, 1):
  print('#i = ', i)
  if( i > 2 ):

    scaler_x_one.fit(x)
    scaler_y_one.fit(y.reshape(-1,1))
    xs1 = scaler_x_one.transform(x)
    ys1 = scaler_y_one.transform(y.reshape(-1,1))
    print('xs1.min = ', xs1.min())
    print('xs1.max = ', xs1.max())
    print('ys1.min = ', ys1.min())
    print('ys1.max = ', ys1.max())
    print('ys1.shape = ', ys1.shape)
    x_scaled = xs1
    y_scaled = ys1

    # faszomat bazmeg, megint kint van skálázva a kurva életbe !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    x_part = x_scaled[:i,:]
    y_part = y_scaled[:i,:]

    mlp.fit(x_part, y_part)
    predicted = mlp.predict(x_part)

    r2 = r2_score(y_part, predicted)
    r2_history.append(r2)

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_part, y_part)
    ax[0].plot(x_part, predicted, c='red')
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i) + ' r2 = {:.3f}'.format(round(r2,3)) + ' loss = {:.3f}'.format(round(mlp.loss_, 3)))
    ax[1].plot(r2_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2,3)))
    ax[1].set_ylim((0, 1.1))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()



from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

## Ugyan ez csak az illeszkedés mértékét is plottolja

## Belül van normálva

Ami itt érdekes lehet, mi van ha nem egyesévél léptetem a for ciklust, az adatokat? (1, 2, 5, 10)

In [ ]:
! rm *.png
! rm *.gif


# mindíg az aktuális értéket normáljuk és nem előre

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one = MinMaxScaler(feature_range=(-1,1))

################################################################################
# A lényeg, hogy nehogy  a forciklusba tegyük a példányosítást, mert akkor
# minden egyes alkalommal újra új példány jön létre és elfelejti a korábbi
# súllyokat amiket megtanult
################################################################################

mlp = MLPRegressor(hidden_layer_sizes=(10, 5),
                  activation='tanh',              # tanh, logistic, relu
                  solver='sgd',                   # adam, sgd, lbfgs
                  batch_size='auto',
                  learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                  max_iter=1,                     # Fontos
                  shuffle=False,                  # Fontos
                  random_state=1,
                  warm_start=True,                # Fontos ?
                  momentum=0.9,
                  nesterovs_momentum=False,
                  early_stopping=False,           # Fontos ?
                  verbose=1,
                  n_iter_no_change=1000)

r2_history = [0]

for i in range(0, 500, 1):
  print('#i = ', i)
  if( i > 2 ):

    # faszomat bazmeg, na most már bent van skálázva !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

    print(x.shape)
    print(y.shape)

    x_part = x[:i,:]
    y_part = y[:i]

    scaler_x_one.fit(x_part)
    scaler_y_one.fit(y_part.reshape(-1,1))
    xs1 = scaler_x_one.transform(x_part)
    ys1 = scaler_y_one.transform(y_part.reshape(-1,1))
    print('xs1.min = ', xs1.min())
    print('xs1.max = ', xs1.max())
    print('ys1.min = ', ys1.min())
    print('ys1.max = ', ys1.max())
    print('ys1.shape = ', ys1.shape)
    x_scaled = xs1
    y_scaled = ys1


    mlp.fit(x_scaled, y_scaled)
    predicted = mlp.predict(x_scaled)

    r2 = r2_score(y_scaled, predicted)
    r2_history.append(r2)

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted, c='red')
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i) + ' r2 = {:.3f}'.format(round(r2,3)) + ' loss = {:.3f}'.format(round(mlp.loss_, 3)))
    ax[1].plot(r2_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2,3)))
    ax[1].set_ylim((0, 1.1))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()



from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

In [ ]:
plt.plot(mlp.loss_curve_)

## Van egy úgynevezett zseniális ötletem

Az merült ma fel bennem, hogy mi lenne akkor ha online módszerrel folyamatosan<br>
tanítanánk a neurális hálót, de nem csak egyet, hanem rögtön többet is<br>
különböző learning rate-tel.<br>

Ennek az a lényege, hogy a tanulás egyes pontjain a magasabb learning rate<br>
miatt néha 'ugrik' egyet a becslés az egyel korábbihoz képest és lehet,<br>
hogy egy másik learning rate-tel tanított háló 'abszolút hibája' vagy<br>
'négyzetes hibája' alapján közelebb áll a becslése a valós értékehez<br>
ezért mindíg azt választjuk ki egy adott időpontban amelyik becslése<br>
pontosabb eredményt ad.<br>

Ilyen módon a 'hiper paramétert' is menet közben választjuk ki<br>

Van ennek egyébként egy tágabbb elméleti megfontolása is<br>

Mivel incrementált vagyis online tanításról van szó mehet közben<br>
meggondolhatjuk magunkat és fenn áll annak a lehetősége, hogy<br>
másik hiperparaméterrel kellett volna tanítani a hálót.<br>
Csakhogy ezen menet közben már nem lehet változtatni ezért azt csinálom<br>
hogy egyszerre több neurális hálót kezdek el tanítani különböző<br>
hiperparaméterekkel és mindíg azt választom amelyik jobb becslést ad.<br>

Egy picit olyan post-hoc hipotézis vizsgálathoz hasonlít ez mint, ha<br>
azt mondom, hogy az volt a feltételezésem, hogy de kiderült, hogy.<br>

Tehát élünk annak a lehetőségével, hogy módosítsunk egy korábbi feltételezésünkön<br>
ami még mindíg jobb mintha ragaszkodnánk ahhoz, amiről már tudjuk, hogy rossz.

In [ ]:
! rm *.png
! rm *.gif


# mindíg az aktuális értéket normáljuk és nem előre

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one = MinMaxScaler(feature_range=(-1,1))

################################################################################
# A lényeg, hogy nehogy  a forciklusba tegyük a példányosítást, mert akkor
# minden egyes alkalommal újra új példány jön létre és elfelejti a korábbi
# súllyokat amiket megtanult
################################################################################

mlp = MLPRegressor(hidden_layer_sizes=(10, 5),
                  activation='tanh',              # tanh, logistic, relu
                  solver='sgd',                   # adam, sgd, lbfgs
                  batch_size='auto',
                  learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                  max_iter=1,                     # Fontos
                  shuffle=False,                  # Fontos
                  random_state=1,
                  warm_start=True,                # Fontos ?
                  momentum=0.9,
                  nesterovs_momentum=False,
                  early_stopping=False,           # Fontos ?
                  verbose=1,
                  n_iter_no_change=1000)

r2_history = [0]

for i in range(0, 500, 1):
  print('#i = ', i)
  if( i > 2 ):

    x_part = x[:i,:]
    y_part = y[:i]

    scaler_x_one.fit(x_part)
    scaler_y_one.fit(y_part.reshape(-1,1))
    xs1 = scaler_x_one.transform(x_part)
    ys1 = scaler_y_one.transform(y_part.reshape(-1,1))
    x_scaled = xs1
    y_scaled = ys1

    x_scaled = scaler_x_one.fit_transform(x_part)
    y_scaled = scaler_y_one.fit_transform(y_part.reshape(-1,1)).reshape(-1,1)



    mlp.fit(x_scaled, y_scaled)
    predicted = mlp.predict(x_scaled)

    r2 = r2_score(y_scaled, predicted)
    r2_history.append(r2)

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted, c='red')
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i) + ' r2 = {:.3f}'.format(round(r2,3)) + ' loss = {:.3f}'.format(round(mlp.loss_, 3)))
    ax[1].plot(r2_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2,3)))
    ax[1].set_ylim((0, 1.1))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()



from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

## ToDo

Két dolgot kéne itt kipróbálni

(A) az MLPRegressorn belül az iterációk számál lehet játszani

(B) a for ciklus lépésközeivel lehet játszani


(C) természetesen a learning ratetel is lehet játszani


(D) és a zseniális ötletemet sem szabad elvetni, hogy egyszerre több modelt tanítunk és minden egyes lépésben azt választjuk amelyik a legjobb eredményt adja éppen akkor

(E) ki szeretném próbálni azt is, hogy a partial_fit metodust hívom úgy, hogy
az összes adatot odaadom neki, 1 iteráció van megengedve, de ezt meghívom 2000-szer és megnézem, hogy azonos eredményt adott-e mintha a sima fit() metodust hívom 2000 max_iter paraméterrel (természetesen a folyamatot is monitorozni kell és összehasonlítani)

(F) !

Illetve van még egy olyan kísérlet, hogy a partial_fit, vagy a fit() hívásakor csak az új adatot adom neki oda és nem a megnövelt adatot (!) van különgég. Az egyik esetben csak egy adatot kap, a második esetben mindig n + 1 adatot.



## ToDo (E)

ki szeretném próbálni azt is, hogy a partial_fit metodust hívom úgy, hogy az összes adatot odaadom neki, 1 iteráció van megengedve, de ezt meghívom 2000-szer és megnézem, hogy azonos eredményt adott-e mintha a sima fit() metodust hívom 2000 max_iter paraméterrel (természetesen a folyamatot is monitorozni kell és összehasonlítani)

Tehát a lényeg, hogy
- mindíg megkapja az összes adatot
- de egyszerre csak egy iterációt csinál
- viszont ezt úgy, hogy nem felejt -> incremental learning
- végül összehasonlítaom azzal, ha a max_iter = 2000

In [ ]:
! rm *.png
! rm *.gif

# Mivel minden adatot odadunk ezért egy lépésben normalizálhatjuk az egészet

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one = MinMaxScaler(feature_range=(-1,1))

x_scaled = scaler_x_one.fit_transform(x)
y_scaled = scaler_y_one.fit_transform(y.reshape(-1,1)).reshape(-1,1)

print('x_scaled.shape = ', x_scaled.shape)
print('y_scaled.shape = ', y_scaled.shape)
print('x_scaled.min   = ', x_scaled.min())
print('x_scaled.max   = ', x_scaled.max())
print('y_scaled.min   = ', y_scaled.min())
print('y_scaled.max   = ', y_scaled.max())

################################################################################
# A lényeg, hogy nehogy  a forciklusba tegyük a példányosítást, mert akkor
# minden egyes alkalommal újra új példány jön létre és elfelejti a korábbi
# súllyokat amiket megtanult
################################################################################

mlp_online = MLPRegressor(hidden_layer_sizes=(10, 5),
                          activation='tanh',              # tanh, logistic, relu
                          solver='sgd',                   # adam, sgd, lbfgs
                          batch_size='auto',
                          learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                          max_iter=1,                     # Mindíg csak egy iterációt végzünk el
                          shuffle=False,                  # Fontos
                          random_state=1,
                          warm_start=True,                # Fontos
                          momentum=0.9,
                          nesterovs_momentum=False,
                          early_stopping=False,           # Fontos
                          verbose=0,
                          n_iter_no_change=1000)

r2_online_history = [0]

for i in range(0, 1000, 1):
  # print('#i = ', i)

  mlp_online.fit(x_scaled, y_scaled)
  predicted_online = mlp_online.predict(x_scaled)

  r2_online = r2_score(y_scaled, predicted_online)
  r2_online_history.append(r2_online)

  error_online = mean_squared_error(y_scaled, predicted_online)

  if( i % 10 == 0 and i > 0 ):

    # Néhány lépésenként nézzük meg, hogy milyen eredményt ad neurális háló
    # ha batchben tanítjuk. Ez a háló mindíg újra tanul, és max_iter = i
    # számú lépést fog végrehajtani

    mlp_batch  = MLPRegressor(hidden_layer_sizes=(10, 5),
                              activation='tanh',              # tanh, logistic, relu
                              solver='sgd',                   # adam, sgd, lbfgs
                              batch_size='auto',
                              learning_rate_init=0.1,         # Fontos 0.01, 0.02, 0.05, 0.1
                              max_iter=i,                     # Az iterációk számát itt a for ciklusból kapjuk
                              shuffle=False,                  # Fontos
                              random_state=1,
                              warm_start=False,               # Fontos
                              momentum=0.9,
                              nesterovs_momentum=False,
                              early_stopping=False,
                              verbose=0,
                              n_iter_no_change=1000)
    
    mlp_batch.fit(x_scaled, y_scaled)
    predicted_batch = mlp_batch.predict(x_scaled)

    error_batch = mean_squared_error(y_scaled, predicted_batch)


    # Plottoljuk ki az eredményeket

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted_online, c='red', label='online error = ' + str(round(error_online, 3)))
    ax[0].plot(x_scaled, predicted_batch, c='black', label='batch error  = ' + str(round(error_batch, 3)))
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i) + ' r2 = {:.3f}'.format(round(r2_online,3)) + ' loss = {:.3f}'.format(round(mlp_online.loss_, 3)))
    ax[1].plot(r2_online_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2_online,3)))
    ax[1].set_ylim((0, 1.1))
    ax[0].legend(loc=3)
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()


# Fűzzük össze a képeket

from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

# Create images

Ezt csak érdeksségnek raktam ide, kétféle képen ábrázolja a hiba (helyesebben) az r2 alakulását.

In [ ]:
! rm *.png
! rm *.gif

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1

r2_history = [0]

for i in range(0, 150, 1):
  print('#i = ', i)
  if( i != 0 ):
    mlp = MLPRegressor(hidden_layer_sizes=(30, 15),
                      activation='tanh', 
                      solver='adam',
                      batch_size='auto',
                      learning_rate_init=0.01,
                      max_iter=i,
                      shuffle=True,
                      random_state=1,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      n_iter_no_change=2000)

    mlp.fit(x_scaled, y_scaled)
    predicted = mlp.predict(x_scaled)

    r2 = r2_score(y_scaled, predicted)
    r2_history.append(r2)

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted, c='red')
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i))
    ax[1].plot(r2_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2,3)))
    ax[1].set_ylim((0, 1.1))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()

from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)

# Create images

Ezt csak érdeksségnek raktam ide, kétféle képen ábrázolja az r2 alakulását futó ablakban.

In [ ]:
! rm *.png
! rm *.gif

scaler_x_one = MinMaxScaler(feature_range=(-1,1))
scaler_x_one.fit(x)
xs1 = scaler_x_one.transform(x)
scaler_y_one = MinMaxScaler(feature_range=(-1,1))
scaler_y_one.fit(y.reshape(-1,1))
ys1 = scaler_y_one.transform(y.reshape(-1,1))
print('xs1.min = ', xs1.min())
print('xs1.max = ', xs1.max())
print('ys1.min = ', ys1.min())
print('ys1.max = ', ys1.max())
print('ys1.shape = ', ys1.shape)
x_scaled = xs1
y_scaled = ys1

r2_history = [0]


n = 150
for i in range(0, n, 1):
  print('#i = ', i)
  if( i != 0 ):
    mlp = MLPRegressor(hidden_layer_sizes=(30, 15),
                      activation='tanh', 
                      solver='adam',
                      batch_size='auto',
                      learning_rate_init=0.01,
                      max_iter=i,
                      shuffle=True,
                      random_state=1,
                      momentum=0.9,
                      nesterovs_momentum=True,
                      early_stopping=True,
                      n_iter_no_change=2000)

    mlp.fit(x_scaled, y_scaled)
    predicted = mlp.predict(x_scaled)

    r2 = r2_score(y_scaled, predicted)
    r2_history.append(r2)

    fig, ax = plt.subplots(1,2, figsize=(15,4), sharey=False)
    ax[0].scatter(x_scaled, y_scaled)
    ax[0].plot(x_scaled, predicted, c='red')
    ax[0].set_ylim((-1.2, 1.2))
    ax[0].set_title('#i = ' + str(i))
    ax[1].plot(r2_history, c='black')
    ax[1].set_title('r2 = {:.3f}'.format(round(r2,3)))
    ax[1].set_ylim((0, 1.1))
    ax[1].set_xlim((0, n))
    plt.savefig('image{0:03}'.format(i)+'.png')
    plt.show()
    plt.close()

from PIL import Image
import glob

frames = []
imgs = glob.glob("*.png")
imgs.sort()
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

frames[0].save('animation.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=100, loop=0)